# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118900e+02     1.556631e+00
 * time: 0.09225606918334961
     1     1.101453e+01     9.738784e-01
 * time: 0.9656589031219482
     2    -1.197137e+01     1.133919e+00
 * time: 1.2272779941558838
     3    -3.416244e+01     7.625954e-01
 * time: 1.7145891189575195
     4    -4.762104e+01     5.386964e-01
 * time: 1.8574090003967285
     5    -5.705477e+01     2.309768e-01
 * time: 2.011059045791626
     6    -5.987842e+01     2.095364e-01
 * time: 2.1280930042266846
     7    -6.101794e+01     9.431211e-02
 * time: 2.2460761070251465
     8    -6.150093e+01     3.965040e-02
 * time: 2.3635740280151367
     9    -6.177319e+01     3.698170e-02
 * time: 2.4742939472198486
    10    -6.190085e+01     2.471805e-02
 * time: 2.5918960571289062
    11    -6.201617e+01     1.879781e-02
 * time: 2.7086129188537598
    12    -6.207688e+01     1.549350e-02
 * time: 2.825937032699585
    13    -6.211445e+01     1.195618e-02
 * time: 2.94203710556

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522653
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666460838
